In [ ]:
"""
YOLOv12 Training Script for Custom Vehicle Dataset
This script sets up the environment and trains a YOLOv12 model on your locally prepared dataset.
"""

# INITIAL SETUP
!pip install ultralytics PyYAML

# Install Flash Attention for YOLOv12
!wget https://github.com/Dao-AILab/flash-attention/releases/download/v2.7.3/flash_attn-2.7.3+cu11torch2.2cxx11abiFALSE-cp310-cp310-linux_x86_64.whl
!pip install flash_attn-2.7.3+cu11torch2.2cxx11abiFALSE-cp310-cp310-linux_x86_64.whl

# Clone YOLOv12 repository
!git clone https://github.com/sunsmarterjie/yolov12.git
%cd yolov12
!pip install -e .

In [ ]:
from ultralytics import YOLO
import os
import yaml
from google.colab import drive
import glob
from IPython.display import Image, display

In [ ]:
# Mount Google Drive to save models and access dataset
drive.mount('/content/drive')
save_dir = '/content/drive/MyDrive/yolov12_training'
os.makedirs(save_dir, exist_ok=True)

In [ ]:
# DATASET SETUP
# Create a local dataset directory
LOCAL_DATASET_PATH = '/content/vehicle_dataset'
os.makedirs(LOCAL_DATASET_PATH, exist_ok=True)

# Path to your vehicle dataset in Google Drive
DRIVE_DATASET_PATH = '/content/drive/MyDrive/vehicle_dataset_split'

# Copy the dataset from Drive to local Colab storage for better performance
print("Copying dataset from Google Drive to local storage...")
!cp -r {DRIVE_DATASET_PATH}/* {LOCAL_DATASET_PATH}/ 2>/dev/null || echo "Using direct dataset path"

# Check if dataset exists, if not prompt for upload
if not os.path.exists(LOCAL_DATASET_PATH + '/dataset.yaml'):
    if os.path.exists(DRIVE_DATASET_PATH + '/dataset.yaml'):
        LOCAL_DATASET_PATH = DRIVE_DATASET_PATH
        print(f"Using dataset directly from Google Drive: {LOCAL_DATASET_PATH}")
    else:
        print("Dataset not found in Google Drive. Please upload your dataset.")
        print("1. Upload dataset.yaml")
        print("2. Upload train, val, test folders with images and labels")
        from google.colab import files

        # Create directory structure
        for split in ["train", "val", "test"]:
            os.makedirs(os.path.join(LOCAL_DATASET_PATH, split, "images"), exist_ok=True)
            os.makedirs(os.path.join(LOCAL_DATASET_PATH, split, "labels"), exist_ok=True)

        print("Please upload dataset.yaml file:")
        uploaded = files.upload()

        for filename in uploaded.keys():
            os.rename(filename, os.path.join(LOCAL_DATASET_PATH, 'dataset.yaml'))


In [ ]:
# Fix the dataset.yaml file - replace Windows paths with Colab paths
data_yaml_path = os.path.join(LOCAL_DATASET_PATH, 'dataset.yaml')
print(f"Fixing dataset.yaml at {data_yaml_path}")

# Read the dataset.yaml
with open(data_yaml_path, 'r') as f:
    yaml_data = yaml.safe_load(f)

# Fix the paths
yaml_data['path'] = LOCAL_DATASET_PATH  # Set the absolute path to the local dataset
yaml_data['train'] = 'train/images'     # Relative paths within the dataset directory
yaml_data['val'] = 'val/images'
yaml_data['test'] = 'test/images'

# Write the corrected yaml back
with open(data_yaml_path, 'w') as f:
    yaml.dump(yaml_data, f, default_flow_style=False)

print("Dataset YAML file fixed with correct paths.")

# Verify dataset structure
print(f"Checking dataset structure at {LOCAL_DATASET_PATH}...")

# Check if images and labels exist
for split in ['train', 'val', 'test']:
    img_dir = os.path.join(LOCAL_DATASET_PATH, split, 'images')
    label_dir = os.path.join(LOCAL_DATASET_PATH, split, 'labels')

    # Count images and labels
    img_count = len(glob.glob(f"{img_dir}/*.*"))
    label_count = len(glob.glob(f"{label_dir}/*.txt"))

    print(f"- {split}: {img_count} images, {label_count} labels")

# Print the final dataset.yaml content
with open(data_yaml_path, 'r') as f:
    print("\nDataset YAML content:")
    print(f.read())


In [ ]:
# Download pre-trained weights
!wget https://github.com/sunsmarterjie/yolov12/releases/download/turbo/yolov12n.pt -P /content/

# Set the training configuration
model = YOLO('/content/yolov12n.pt')  # Load pre-trained model


In [ ]:
# TRAINING CONFIGURATION
# Adjusted for vehicle detection task
epochs = 100             # For small dataset, 100-150 epochs is often sufficient
batch_size = 16          # Smaller batch size for limited VRAM
image_size = 640         # Standard YOLOv12 input size
patience = 20            # Early stopping patience

# Start training
results = model.train(
    data=data_yaml_path,
    epochs=epochs,
    batch=batch_size,
    imgsz=image_size,
    patience=patience,
    scale=0.5,           # Model-specific augmentation parameter
    mosaic=1.0,
    mixup=0.0,
    copy_paste=0.1,
    project=save_dir,
    name='yolov12n_vehicle_detection',
    exist_ok=True,
    pretrained=True,
    optimizer="AdamW",   # Optimizer for attention-based models
    cos_lr=True,         # Cosine learning rate schedule
    lr0=0.001,           # Initial learning rate
    lrf=0.01,            # Final learning rate ratio
    weight_decay=0.0001, # Weight decay
    warmup_epochs=3,     # Warmup epochs
    close_mosaic=10,     # Disable mosaic augmentation in final epochs
    seed=0,              # For reproducibility
    device=0,            # Use GPU 0
)



In [ ]:
# VALIDATE THE TRAINED MODEL
metrics = model.val(data=data_yaml_path)
print(f"Validation metrics: {metrics}")

In [ ]:
# EXPORT THE MODEL
# Get the path of the best weights
weights_dir = os.path.join(save_dir, 'yolov12n_vehicle_detection', 'weights')
best_pt = os.path.join(weights_dir, 'best.pt')

# Export to ONNX format
model.export(format="onnx", imgsz=image_size)  # Creates best.onnx in the same folder

# Print actual paths for both PT and ONNX models
print(f"PyTorch model saved at: {best_pt}")
print(f"ONNX model exported to: {os.path.join(weights_dir, 'best.onnx')}")

In [ ]:
# RUN INFERENCE ON A TEST IMAGE
# Find all test images in the directory
test_image_dir = os.path.join(LOCAL_DATASET_PATH, 'test', 'images')
test_images = glob.glob(f"{test_image_dir}/*.jpg") + glob.glob(f"{test_image_dir}/*.jpeg") + glob.glob(f"{test_image_dir}/*.png")

if test_images:
    # Use the first test image found
    test_image_path = test_images[0]
    print(f"Running inference on: {test_image_path}")

    # Perform inference
    results = model.predict(test_image_path, save=True, conf=0.25)
    print(f"Inference results saved to: {results[0].save_dir}")

    # Display the results directly in Colab
    display(Image(os.path.join(results[0].save_dir, os.path.basename(test_image_path))))
else:
    print(f"No test images found in {test_image_dir}")
    print("Try running inference on a specific image with:")
    print("model.predict('/path/to/your/image.jpg', save=True, conf=0.25)")

print("Training, validation, and export complete!")